In [ ]:
from flask import Flask, render_template, request
from translate import Translator
import speech_recognition as sr
import pyttsx3
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

app = Flask(__name__)
engine = pyttsx3.init("sapi5")
translator= Translator(to_lang="hindi")
listener = sr.Recognizer()
engine.runAndWait()

@app.route('/')
def home():
    say = True
    if say == False:
        pyttsx3.speak('Hello Sir I am Your personal helper')
        pyttsx3.speak('''use mail check command to check spam mails or use Salry check to predict Employee Salary''')
        pyttsx3.speak("and use language translator to translate language")
        pyttsx3.speak("And use home price to ckeck the price of a home")
    return render_template('pannel.html')

@app.route("/",methods=['GET','POST'])
def check():
    if request.method == "POST":
        name = request.form["name"]
        if name == "mail check":
            return render_template("mail_check.html")
        elif name == "salary check":
            return render_template("salary.html")  
        elif name == "language translator":
            return render_template("translator.html")
        elif name == "home price":
            return render_template("homeprice.html")
        else:
            return("please enter a valid response")

@app.route("/homeprice",methods=['GET','POST'])
def homeprice():
    if request.method == "POST":
        area = request.form['area']
        rooms = request.form['rooms']
        balcony = request.form['age']
        df = pd.read_csv("homeprices.csv")
        model = linear_model.LinearRegression()
        model.fit(df.drop('price',axis='columns'),df.price)
        price = model.predict([[area,rooms,balcony]])
        return render_template("homeprice.html",price=price)
        
@app.route("/translate",methods=['GET','POST'])
def translate():
    if request.method == "POST":
        text = request.form['text']
        translatedText = translator.translate(text)
        return render_template("translator.html",translatedText=translatedText)
        
@app.route("/salary",methods=['GET','POST'])
def salary():
    if request.method == "POST":
        experience = request.form['experience']
        df = pd.read_csv("Salary_Data.csv")
        model = linear_model.LinearRegression()
        new_df = df.drop("Salary",axis="columns")
        model.fit(new_df,df.Salary)
        salary = model.predict([[experience]])
        return render_template("salary.html",salary=salary)
    else:
        return("Internal server error")

@app.route('/predict', methods=['POST'])
def predict():
    df = pd.read_csv("spam.csv", encoding="latin-1")
    df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)
    df['label'] = df['class'].map({'ham': 0, 'spam': 1})
    X = df['message']
    y = df['label']
    cv = CountVectorizer()
    X = cv.fit_transform(X)  
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    clf = MultinomialNB()
    clf.fit(X_train, y_train)
    clf.score(X_test, y_test)
    if request.method == 'POST':
        message = request.form['message']
        data = [message]
        vect = cv.transform(data).toarray()
        my_prediction = clf.predict(vect)
    return render_template('mail_check.html', prediction=my_prediction)

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Aug/2022 10:32:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2022 10:32:30] "POST / HTTP/1.1" 200 -
C:\Users\Priyansh Sarswat\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\Priyansh Sarswat\anaconda3\lib\site-packages\sklearn\base.py:566: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  X = check_array(X, **check_params)
127.0.0.1 - - [23/Aug/2022 10:32:34] "POST /homeprice HTTP/1.1" 200 -
